<a href="https://colab.research.google.com/github/ELISA1220/113-2Programming-language/blob/main/HW0410.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [35]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Lbskz2tt_swwPPyWGFZ5jtL2Gz2diER-4cbTHiw8iMs/edit?usp=sharing').sheet1

In [36]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Lbskz2tt_swwPPyWGFZ5jtL2Gz2diER-4cbTHiw8iMs/edit?usp=sharing').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [37]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [38]:
df.head()

,時間戳記,組別,如果你的班上有學習障礙者，可以如何調整評量方式呢？
0,2022/12/1 下午 12:13:02,第四組,有讀寫障礙的學生，在考國文閱讀測驗時「閱讀不流暢」，老師可以怎麼調整評量方式呢？
1,2022/12/1 下午 12:13:09,第二組,有讀寫障礙的學生，在閱讀英文考卷時，產生「易混淆字詞」的情形，老師可以怎麼調整英文評量方式呢？
2,2022/12/1 下午 12:13:13,第六組,有讀寫障礙的學生，在考國文閱讀測驗時「閱讀不流暢」，老師可以怎麼調整評量方式呢？
3,2022/12/1 下午 12:13:32,第八組,有讀寫障礙的學生，在閱讀英文考卷時，產生「易混淆字詞」的情形，老師可以怎麼調整英文評量方式呢？
4,2022/12/1 下午 12:13:43,第一組,有讀寫障礙的學生「無法理解運算符號」，老師可以怎麼調整數學評量方式呢？


In [39]:
df.describe()

,時間戳記,組別,如果你的班上有學習障礙者，可以如何調整評量方式呢？
count,7,7,7
unique,7,7,3
top,2022/12/1 下午 12:13:02,第四組,有讀寫障礙的學生，在閱讀英文考卷時，產生「易混淆字詞」的情形，老師可以怎麼調整英文評量方式呢？
freq,1,1,3


In [43]:
!pip install jieba
#jieba是個套件，功能是可以拆分、辨識中文字
!pip install snownlp
#分析中文文本的情感，比如可以根據文章內容判斷文本是正面還是負面情感，還可以自動生成簡短的摘要。

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=97afe460dad384ac712788d729c28725459be9839235c8669544983482418e1e
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [44]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [46]:
print(dicts.columns)

Index(['時間戳記', '組別', '如果你的班上有學習障礙者，可以如何調整評量方式呢？'], dtype='object')


In [47]:
for _, row in dicts.iterrows():
    text = row['如果你的班上有學習障礙者，可以如何調整評量方式呢？']  # 取出該列的文本
    words = jieba.cut(text)  # 使用jieba進行分詞
    for word in words:
        jieba.add_word(word, freq=5)
        jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.841 seconds.
DEBUG:jieba:Loading model cost 0.841 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [48]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [50]:
print(df.columns)

Index(['時間戳記', '組別', '如果你的班上有學習障礙者，可以如何調整評量方式呢？'], dtype='object')


In [53]:
# 假設 '如果你的班上有學習障礙者，可以如何調整評量方式呢？' 是你想要處理的欄位
df['Chinese_Content'] = df['如果你的班上有學習障礙者，可以如何調整評量方式呢？'].apply(extract_chinese)


In [54]:
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [55]:
df['Chinese_Content']

,Chinese_Content
0,有讀寫障礙的學生在考國文閱讀測驗時閱讀不流暢老師可以怎麼調整評量方式呢
1,有讀寫障礙的學生在閱讀英文考卷時產生易混淆字詞的情形老師可以怎麼調整英文評量方式呢
2,有讀寫障礙的學生在考國文閱讀測驗時閱讀不流暢老師可以怎麼調整評量方式呢
3,有讀寫障礙的學生在閱讀英文考卷時產生易混淆字詞的情形老師可以怎麼調整英文評量方式呢
4,有讀寫障礙的學生無法理解運算符號老師可以怎麼調整數學評量方式呢
5,有讀寫障礙的學生無法理解運算符號老師可以怎麼調整數學評量方式呢
6,有讀寫障礙的學生在閱讀英文考卷時產生易混淆字詞的情形老師可以怎麼調整英文評量方式呢


In [56]:
df['Tokenized']

,Tokenized
0,"[有讀, 寫, 障礙, 的, 學生, 在, 考國文, 閱讀, 測驗, 時, 閱讀, 不流暢,..."
1,"[有讀, 寫, 障礙, 的, 學生, 在, 閱讀, 英文, 考卷, 時, 產生, 易, 混淆..."
2,"[有讀, 寫, 障礙, 的, 學生, 在, 考國文, 閱讀, 測驗, 時, 閱讀, 不流暢,..."
3,"[有讀, 寫, 障礙, 的, 學生, 在, 閱讀, 英文, 考卷, 時, 產生, 易, 混淆..."
4,"[有讀, 寫, 障礙, 的, 學生, 無法, 理解, 運, 算符, 號, 老師, 可以, 怎..."
5,"[有讀, 寫, 障礙, 的, 學生, 無法, 理解, 運, 算符, 號, 老師, 可以, 怎..."
6,"[有讀, 寫, 障礙, 的, 學生, 在, 閱讀, 英文, 考卷, 時, 產生, 易, 混淆..."


In [57]:
word_counts = Counter(all_words)
word_counts

Counter({'有讀': 7,
         '障礙': 7,
         '學生': 7,
         '考國文': 2,
         '閱讀': 7,
         '測驗': 2,
         '不流暢': 2,
         '老師': 7,
         '可以': 7,
         '怎麼': 7,
         '調整': 7,
         '評量': 7,
         '方式': 7,
         '英文': 6,
         '考卷': 3,
         '產生': 3,
         '混淆': 3,
         '字詞': 3,
         '情形': 3,
         '無法': 2,
         '理解': 2,
         '算符': 2,
         '數學': 2})

In [58]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['可以', '怎麼', '因為', '的', '在', '是', '我'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [59]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(10), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前5名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [60]:
word_freq_df

,Word,Frequency
0,有讀,7
1,障礙,7
2,學生,7
3,閱讀,7
4,調整,7
5,方式,7
6,老師,7
7,評量,7
8,英文,6
9,字詞,3


In [61]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 0s (23.7 MB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [62]:
from wordcloud import WordCloud

# 取前 20 個最常出現的詞
top_words = word_freq_df.head(20)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()